In [ ]:
import os
import sys
sys.path.append(os.path.abspath(os.path.pardir))
from argparse import ArgumentParser

import torch
from voltaml.compile import VoltaGPUCompiler
from voltaml.inference import gpu_performance
import torchvision
from voltaml.models.common import DetectMultiBackend
import warnings
warnings.filterwarnings('ignore')

### Load Model 

In [ ]:
torch_model_dir = 'voltaml/yolov5s.pt'
model = DetectMultiBackend(torch_model_dir)

## Set parameters for FP16

In [ ]:
input_shape = (1,3,640,640)
precision = 'fp16'
compiled_model_dir = '/workspace/voltaml/models/yolov5_fp16.engine'
is_yolo = True
input_name = 'images'
output_name = 'output'
simplify = True

### Compile Model

In [ ]:
compiler = VoltaGPUCompiler(
    model=model,
    output_dir=compiled_model_dir,
    input_shape=input_shape,
    precision=precision,
    input_name=input_name,
    output_name=output_name,
    simplify=simplify
    
)

compiled_model = compiler.compile()

In [ ]:
gpu_performance(compiled_model_dir, model, input_shape=input_shape, is_yolo=is_yolo)

### Set parameters for INT8

In [ ]:
input_shape = (1,3,224,224)
precision = 'int8'
compiled_model_dir = '/workspace/voltaml/models/resnet50_int8.engine'
throughput_batch_size = 1
calib_input = '/workspace/TensorRT/ILSVRC2012_img_val/'
calib_cache = 'models/calibration.cache'
calib_num_images=25000
calib_batch_size=8
calib_preprocessor='V2'

### Compile Model

In [ ]:
compiler = VoltaGPUCompiler(
    model=model,
    output_dir=compiled_model_dir,
    input_shape=input_shape,
    precision=precision,
    calib_input=calib_input,
    calib_cache=calib_cache,
    calib_num_images=calib_num_images,
    calib_batch_size=calib_batch_size,
    calib_preprocessor=calib_preprocessor
)

compiled_model = compiler.compile()

In [ ]:
gpu_performance(compiled_model_dir, model, input_shape=input_shape, throughput_batch_size=throughput_batch_size)